Part 1: Data Preprocessing

In [102]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import pandas as pd
import gzip

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [103]:
with gzip.open(r"C:\Users\snuga\OneDrive\Documents\USF\Research\MIMIC_IV\admissions.csv.gz", 'rb') as file:
    admissionsdf = pd.read_csv(file)

admissionsdf

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [104]:
with gzip.open(r"C:\Users\snuga\OneDrive\Documents\USF\Research\MIMIC_IV\diagnoses_icd.csv.gz", 'rb') as file:
    diagnosesdf = pd.read_csv(file)

diagnosesdf

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [105]:
with gzip.open(r"C:\Users\snuga\OneDrive\Documents\USF\Research\MIMIC_IV\d_icd_diagnoses.csv.gz", 'rb') as file:
    diagnosisdescriptionsdf = pd.read_csv(file)

diagnosisdescriptionsdf

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [106]:
joineddataset=pd.merge(admissionsdf,diagnosesdf, on=['subject_id', 'hadm_id'])
joineddataset

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,1,5723,9
1,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,2,78959,9
2,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,3,5715,9
3,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,4,07070,9
4,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,5,496,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756321,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,ENGLISH,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,7,41401,9
4756322,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,ENGLISH,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,8,78039,9
4756323,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,ENGLISH,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,9,0413,9
4756324,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,ENGLISH,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,10,36846,9


In [107]:
with gzip.open(r"C:\Users\snuga\OneDrive\Documents\USF\Research\MIMIC_IV\patients.csv.gz", 'rb') as file:
    patientsdf = pd.read_csv(file)

patientsdf

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN
299708,19999829,F,28,2186,2008 - 2010,NaN
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,NaN


In [108]:
joineddataset2 = pd.merge(joineddataset,patientsdf[['subject_id','anchor_age', 'gender']], on='subject_id', how='left')
joineddataset2

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version,anchor_age,gender
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,1,5723,9,52,F
1,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,2,78959,9,52,F
2,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,3,5715,9,52,F
3,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,4,07070,9,52,F
4,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,5,496,9,52,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756321,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,7,41401,9,57,F
4756322,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,8,78039,9,57,F
4756323,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,9,0413,9,57,F
4756324,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,10,36846,9,57,F


In [109]:
joinedandsorteddf=joineddataset2.sort_values(by=['subject_id', 'admittime', 'seq_num'], ascending=True)
joinedandsorteddf

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version,anchor_age,gender
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,1,5723,9,52,F
1,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,2,78959,9,52,F
2,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,3,5715,9,52,F
3,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,4,07070,9,52,F
4,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,5,496,9,52,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756321,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,7,41401,9,57,F
4756322,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,8,78039,9,57,F
4756323,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,9,0413,9,57,F
4756324,19999987,23865745,2145-11-02 21:38:00,2145-11-11 12:57:00,NaN,EW EMER.,P09IS0,EMERGENCY ROOM,REHAB,Other,...,NaN,UNKNOWN,2145-11-02 19:28:00,2145-11-02 22:59:00,0,10,36846,9,57,F


In [110]:
patientcount=joinedandsorteddf.subject_id.nunique()
patientcount

180640

In [111]:
#Filter Based on ICD-10 Code
joinedandsorteddf=joinedandsorteddf[joinedandsorteddf["icd_version"]==10]
joinedandsorteddf

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version,anchor_age,gender
40,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,1,G3183,10,72,M
41,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,2,F0280,10,72,M
42,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,3,R441,10,72,M
43,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,4,R296,10,72,M
44,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,5,E785,10,72,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756269,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0,15,F419,10,46,F
4756270,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0,16,E876,10,46,F
4756271,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0,17,B954,10,46,F
4756272,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0,18,E60,10,46,F


In [112]:
averageage=joinedandsorteddf.groupby('race', as_index=False)['anchor_age'].mean().sort_values(by=['anchor_age'])
averageage

,race,anchor_age
17,HISPANIC/LATINO - MEXICAN,48.819292
29,WHITE - BRAZILIAN,50.088816
4,ASIAN - KOREAN,52.022636
14,HISPANIC/LATINO - DOMINICAN,52.263907
19,HISPANIC/LATINO - SALVADORAN,52.396809
15,HISPANIC/LATINO - GUATEMALAN,52.927043
18,HISPANIC/LATINO - PUERTO RICAN,53.913656
16,HISPANIC/LATINO - HONDURAN,54.049141
6,BLACK/AFRICAN,54.925305
7,BLACK/AFRICAN AMERICAN,55.895933


In [113]:
#Total Number of ICD-10 Codes
icdcount=joinedandsorteddf.icd_code.nunique()
icdcount

16757

In [114]:
#Most Frequent ICD-10 Codes

icdgroupcount=joinedandsorteddf.groupby(['icd_code'])['subject_id'].nunique().sort_values(ascending=False)
icdgroupcount=pd.DataFrame(icdgroupcount)
icdgroupcount = icdgroupcount.merge(diagnosisdescriptionsdf, on='icd_code', how='left')

In [115]:
sample2=icdgroupcount.head(10)
sample2

,icd_code,subject_id,icd_version,long_title
0,I10,31521,10,Essential (primary) hypertension
1,E785,27903,10,"Hyperlipidemia, unspecified"
2,Z87891,21356,10,Personal history of nicotine dependence
3,K219,19067,10,Gastro-esophageal reflux disease without esoph...
4,F329,16476,10,"Major depressive disorder, single episode, uns..."
5,F419,14223,10,"Anxiety disorder, unspecified"
6,I2510,13438,10,Atherosclerotic heart disease of native corona...
7,N179,13394,10,"Acute kidney failure, unspecified"
8,Y929,11706,10,Unspecified place or not applicable
9,E119,10302,10,Type 2 diabetes mellitus without complications


In [116]:
#Total Number of Patients

patientcount=joinedandsorteddf.subject_id.nunique()
patientcount

80213

In [117]:
#Deceased Patient Count

deceasedpatientcounts=joinedandsorteddf[joinedandsorteddf["deathtime"].notnull()]["subject_id"].nunique()
deceasedpatientcounts

3360

In [118]:
#Dataframe just based on Deceased Patients

deceasedpatients=joinedandsorteddf[joinedandsorteddf["deathtime"].notnull()]
deceasedpatients

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version,anchor_age,gender
750,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1,1,J441,10,68,F
751,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1,2,K7200,10,68,F
752,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1,3,R579,10,68,F
753,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1,4,J9602,10,68,F
754,10001884,26184834,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1,5,J9601,10,68,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4751631,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,24,T24011A,10,67,F
4751632,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,25,Y658,10,67,F
4751633,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,26,Y92230,10,67,F
4751634,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,27,H168,10,67,F


In [119]:
#List of Deceased Patients
arrayofdeceasedpatients=deceasedpatients["subject_id"].unique().copy()
listofdeceasedpatients=arrayofdeceasedpatients.tolist()
listofdeceasedpatients

[10001884,
 10003400,
 10004720,
 10005817,
 10007818,
 10010058,
 10010471,
 10011365,
 10015931,
 10021927,
 10023117,
 10024982,
 10026879,
 10032409,
 10035631,
 10037861,
 10038688,
 10046679,
 10047172,
 10050445,
 10050755,
 10055344,
 10055694,
 10059406,
 10064049,
 10067389,
 10069692,
 10071869,
 10073239,
 10076958,
 10078805,
 10079545,
 10080695,
 10082662,
 10097659,
 10106165,
 10113636,
 10115044,
 10118141,
 10119017,
 10119992,
 10131638,
 10132611,
 10133363,
 10133599,
 10134367,
 10135376,
 10142197,
 10144378,
 10152346,
 10155871,
 10157542,
 10162760,
 10168722,
 10177348,
 10179383,
 10179842,
 10183369,
 10184327,
 10187075,
 10192651,
 10197095,
 10201643,
 10206125,
 10207354,
 10213338,
 10236222,
 10236280,
 10236942,
 10237748,
 10239650,
 10240102,
 10246238,
 10249699,
 10250007,
 10252642,
 10257550,
 10260379,
 10261293,
 10263326,
 10269234,
 10272082,
 10272169,
 10274368,
 10284297,
 10289851,
 10291949,
 10292730,
 10295734,
 10300273,
 10302157,

In [120]:
#See if it Checks out

len(listofdeceasedpatients)

#yes

3360

In [121]:
#Now filter original dataframe based on the dead patients list

joinedandsorteddf2=joinedandsorteddf[joinedandsorteddf['subject_id'].isin(listofdeceasedpatients)]
joinedandsorteddf2

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,seq_num,icd_code,icd_version,anchor_age,gender
819,10001884,27016754,2130-06-23 22:14:00,2130-06-24 11:07:00,NaN,EU OBSERVATION,P21E4S,EMERGENCY ROOM,NaN,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2130-06-23 17:42:00,2130-06-24 11:07:00,0,1,J441,10,68,F
821,10001884,27016754,2130-06-23 22:14:00,2130-06-24 11:07:00,NaN,EU OBSERVATION,P21E4S,EMERGENCY ROOM,NaN,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2130-06-23 17:42:00,2130-06-24 11:07:00,0,2,F17210,10,68,F
823,10001884,27016754,2130-06-23 22:14:00,2130-06-24 11:07:00,NaN,EU OBSERVATION,P21E4S,EMERGENCY ROOM,NaN,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2130-06-23 17:42:00,2130-06-24 11:07:00,0,3,R079,10,68,F
786,10001884,26202981,2130-08-21 15:26:00,2130-08-23 16:40:00,NaN,EW EMER.,P20301,EMERGENCY ROOM,HOME,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2130-08-21 12:14:00,2130-08-21 16:51:00,0,1,K921,10,68,F
787,10001884,26202981,2130-08-21 15:26:00,2130-08-23 16:40:00,NaN,EW EMER.,P20301,EMERGENCY ROOM,HOME,Medicare,...,MARRIED,BLACK/AFRICAN AMERICAN,2130-08-21 12:14:00,2130-08-21 16:51:00,0,2,D62,10,68,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4751631,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,24,T24011A,10,67,F
4751632,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,25,Y658,10,67,F
4751633,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,26,Y92230,10,67,F
4751634,19990427,29695607,2182-01-04 00:00:00,2182-01-24 00:00:00,2182-01-24 23:15:00,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,DIED,Medicare,...,MARRIED,WHITE,NaN,NaN,1,27,H168,10,67,F


In [122]:
#Group by the admittime and the subject_id (either df of all patients or deceased patients)

groupedlist=joinedandsorteddf2.groupby(["subject_id", "admittime"], as_index=False)["icd_code"].apply(list)
groupedlist

,subject_id,admittime,icd_code
0,10001884,2130-06-23 22:14:00,"[J441, F17210, R079]"
1,10001884,2130-08-21 15:26:00,"[K921, D62, I4891, J449, I10, F17210, D649, I2..."
2,10001884,2130-10-05 20:04:00,"[J441, R0902, E876, I10, E780, E785, Z87891, Z..."
3,10001884,2130-10-08 19:03:00,"[I4891, J441, Z9981, Z7901, I10, E785, F419, I..."
4,10001884,2130-10-14 00:58:00,"[J441, Z9981, I10, I4891, Z7901, Z87891]"
...,...,...,...
8549,19984052,2184-11-22 17:00:00,"[I82412, N179, I498, Z951, I469, I82442, I10, ..."
8550,19984875,2118-01-05 00:18:00,"[C7931, G936, G9340, Z85118, F419, D509, Z902,..."
8551,19984875,2118-02-04 19:36:00,"[C7931, G936, G9349, G8191, G40109, R64, Z6826..."
8552,19984875,2118-04-18 21:31:00,"[C7931, G936, J9691, R4701, Z515, C3491, G8191..."


In [123]:
#drop admittime
groupedlist = groupedlist.drop(columns=['admittime']).copy()
groupedlist

,subject_id,icd_code
0,10001884,"[J441, F17210, R079]"
1,10001884,"[K921, D62, I4891, J449, I10, F17210, D649, I2..."
2,10001884,"[J441, R0902, E876, I10, E780, E785, Z87891, Z..."
3,10001884,"[I4891, J441, Z9981, Z7901, I10, E785, F419, I..."
4,10001884,"[J441, Z9981, I10, I4891, Z7901, Z87891]"
...,...,...
8549,19984052,"[I82412, N179, I498, Z951, I469, I82442, I10, ..."
8550,19984875,"[C7931, G936, G9340, Z85118, F419, D509, Z902,..."
8551,19984875,"[C7931, G936, G9349, G8191, G40109, R64, Z6826..."
8552,19984875,"[C7931, G936, J9691, R4701, Z515, C3491, G8191..."


In [124]:
#Convert to string

groupedlist['icd_code'] = groupedlist['icd_code'].astype(str)
groupedlist

,subject_id,icd_code
0,10001884,"['J441', 'F17210', 'R079']"
1,10001884,"['K921', 'D62', 'I4891', 'J449', 'I10', 'F1721..."
2,10001884,"['J441', 'R0902', 'E876', 'I10', 'E780', 'E785..."
3,10001884,"['I4891', 'J441', 'Z9981', 'Z7901', 'I10', 'E7..."
4,10001884,"['J441', 'Z9981', 'I10', 'I4891', 'Z7901', 'Z8..."
...,...,...
8549,19984052,"['I82412', 'N179', 'I498', 'Z951', 'I469', 'I8..."
8550,19984875,"['C7931', 'G936', 'G9340', 'Z85118', 'F419', '..."
8551,19984875,"['C7931', 'G936', 'G9349', 'G8191', 'G40109', ..."
8552,19984875,"['C7931', 'G936', 'J9691', 'R4701', 'Z515', 'C..."


In [125]:
#Filter for Just the Patients With Cardiac Arrest in Final Sequence

newgroupedlist=groupedlist.drop_duplicates('subject_id',keep='last').\
           assign(Flag=lambda x : x['icd_code'].str.contains('I462|I468|I469').astype(int))



newgroupedlist=newgroupedlist[newgroupedlist['Flag']==1]


arrayofdeceasedcardiacarrestpatients=newgroupedlist["subject_id"].unique().copy()
listofdeceasedcardiacarrestpatients=arrayofdeceasedcardiacarrestpatients.tolist()
print(len(listofdeceasedcardiacarrestpatients))

363


In [126]:
#All sequences for patients where cardiac arrest contributed to death
targetpatients=groupedlist[groupedlist['subject_id'].isin(listofdeceasedcardiacarrestpatients)]
targetpatients

,subject_id,icd_code
0,10001884,"['J441', 'F17210', 'R079']"
1,10001884,"['K921', 'D62', 'I4891', 'J449', 'I10', 'F1721..."
2,10001884,"['J441', 'R0902', 'E876', 'I10', 'E780', 'E785..."
3,10001884,"['I4891', 'J441', 'Z9981', 'Z7901', 'I10', 'E7..."
4,10001884,"['J441', 'Z9981', 'I10', 'I4891', 'Z7901', 'Z8..."
...,...,...
8526,19942414,"['I748', 'I5033', 'A419', 'R6521', 'N280', 'N1..."
8528,19948220,"['I469', 'R4020', 'J9601', 'J930', 'E872', 'I4..."
8548,19984052,"['I2510', 'D696', 'E119', 'E669', 'Z794', 'I10..."
8549,19984052,"['I82412', 'N179', 'I498', 'Z951', 'I469', 'I8..."


In [127]:
#Aggregate to one row

newtargetpatients = (targetpatients.groupby(['subject_id'])
       .agg({'icd_code': lambda x: ', '.join(x)})
       .reset_index()
)

newtargetpatients

,subject_id,icd_code
0,10001884,"['J441', 'F17210', 'R079'], ['K921', 'D62', 'I..."
1,10010471,"['I4891', 'N186', 'J189', 'Z7682', 'I248', 'J4..."
2,10024982,"['I25110', 'Z951', 'I4891', 'J449', 'Z87891', ..."
3,10050755,"['I63132', 'J189', 'J690', 'E43', 'N179', 'G92..."
4,10067389,"['J690', 'L89153', 'I639', 'N170', 'J9601', 'K..."
...,...,...
358,19940062,"['G935', 'G950', 'I252', 'N400', 'V499XXS', 'Z..."
359,19942414,"['I748', 'I5033', 'A419', 'R6521', 'N280', 'N1..."
360,19948220,"['I469', 'R4020', 'J9601', 'J930', 'E872', 'I4..."
361,19984052,"['I2510', 'D696', 'E119', 'E669', 'Z794', 'I10..."


In [128]:
#Get counts of sequences

countlist=[]
for x in newtargetpatients['icd_code']:
    countlist.append(x.count(",")+1)
print(max(countlist), min(countlist), sum(countlist))

409 6 17379


In [129]:
newtargetpatients=newtargetpatients['icd_code'].values.tolist()
newtargetpatients

["['J441', 'F17210', 'R079'], ['K921', 'D62', 'I4891', 'J449', 'I10', 'F17210', 'D649', 'I2510', 'F419', 'E785', 'I739', 'M47892', 'R0602', 'G4700', 'H409', 'K449', 'K2970', 'Z7982', 'Z7901', 'Z96649'], ['J441', 'R0902', 'E876', 'I10', 'E780', 'E785', 'Z87891', 'Z9981'], ['I4891', 'J441', 'Z9981', 'Z7901', 'I10', 'E785', 'F419', 'I739', 'I2510', 'Z87891', 'D509', 'R079', 'G4700', 'H409'], ['J441', 'Z9981', 'I10', 'I4891', 'Z7901', 'Z87891'], ['J441', 'I4892', 'Z9981', 'I480', 'J45998', 'Z87891', 'I2510', 'Z96649', 'I10', 'E785', 'D509', 'I739', 'F419', 'K5900', 'M1990', 'Z825', 'Z8249', 'Z7901'], ['J441', 'M7989', 'I2510', 'I499', 'I739', 'F419', 'Z87891', 'E876'], ['J441', 'I4892', 'I248', 'J45909', 'Z87891', 'I4891', 'F419', 'G4700', 'E780', 'I2510', 'E876', 'R312', 'I739', 'Z7952', 'Z9981'], ['J441', 'Z9981', 'I4891', 'J45909', 'Z7901', 'I10', 'I2510', 'E785', 'M1990', 'F419', 'I739', 'G4700', 'D649', 'Z96649', 'Z87891'], ['J441', 'J45909', 'H6991', 'F419', 'G4700', 'I4891', 'Z7901'

In [130]:
from ast import literal_eval

example=newtargetpatients

result2=[]
for st in example:
    *first_grp,last=literal_eval(st)
    if any(isinstance(s, str) for s in (first_grp, last)):
        continue
    result=[" ".join([e for r in first_grp for e in r]), " ".join(last)]
    result2.append(result)

print(result2)

[['J441 F17210 R079 K921 D62 I4891 J449 I10 F17210 D649 I2510 F419 E785 I739 M47892 R0602 G4700 H409 K449 K2970 Z7982 Z7901 Z96649 J441 R0902 E876 I10 E780 E785 Z87891 Z9981 I4891 J441 Z9981 Z7901 I10 E785 F419 I739 I2510 Z87891 D509 R079 G4700 H409 J441 Z9981 I10 I4891 Z7901 Z87891 J441 I4892 Z9981 I480 J45998 Z87891 I2510 Z96649 I10 E785 D509 I739 F419 K5900 M1990 Z825 Z8249 Z7901 J441 M7989 I2510 I499 I739 F419 Z87891 E876 J441 I4892 I248 J45909 Z87891 I4891 F419 G4700 E780 I2510 E876 R312 I739 Z7952 Z9981 J441 Z9981 I4891 J45909 Z7901 I10 I2510 E785 M1990 F419 I739 G4700 D649 Z96649 Z87891 J441 J45909 H6991 F419 G4700 I4891 Z7901 I10 I2510 D649 E785 Z87891 Z006 J441 N179 Z9981 I4891 D649 I10 E785 G5622 I2510 M1990 Z96649 Z87891 J45909 F419 G4700 R040 I739', 'J441 K7200 R579 J9602 J9601 I442 I82621 I4891 D696 I469 I10 E785 Z7901 M47892 I7389 I2510 Z87891 Z96641 J0190 F419 G4700 R609 D509 R509 M1990 Y92239 Z7952 Z825 Z781 Z515 Z66 Z9981 J45909 I447 M7981 T45515A'], ['I4891 N186 J189 

Start Model

In [131]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [132]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    #lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        #read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = result2

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [133]:
MAX_LENGTH = 250

def filterPair(p):
    return len(p[0]) < MAX_LENGTH and \
        len(p[1]) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [134]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('prior', 'last', False)
print(random.choice(pairs))

Reading lines...
Read 140 sentence pairs
Trimmed to 87 sentence pairs
Counting words...
Counted words:
prior 716
last 733
['E8770 N186 Z7682 I120 E1122 I272 E785 E875 F329 E11349 E669 R0902 D649 H905 D72829 J984 E11649 T380X5A K5900 Z794 Z992 Z87891 Z6834', 'I2699 D65 J9602 I213 I468 I472 E1122 D62 N186 I132 I82411 E872 I82441 I824Z1 I82532 I5032 E2740 R578 E113499 I2720 I2510 Z992 R569 G4733 Z87891 D631 R740 E785 E669 I071 Z794 E1165 Z6832']


In [135]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [136]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [137]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [138]:
from sklearn.model_selection import train_test_split

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size, test_or_train_data):
    input_lang, output_lang, pairs = prepareData('prior', 'last', False)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids
    
        input_ids_train, input_ids_test, target_ids_train, target_ids_test = train_test_split(input_ids, target_ids, test_size=0.20, random_state=42)

    train_data = TensorDataset(torch.LongTensor(input_ids_train).to(device),
                               torch.LongTensor(target_ids_train).to(device))
    
    global test_data
    test_data = TensorDataset(torch.LongTensor(input_ids_test).to(device),
                               torch.LongTensor(target_ids_test).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    
    if test_or_train_data == train1:
        return input_lang, output_lang, train_dataloader
    if test_or_train_data == train2:
        return input_lang, output_lang, train_dataloader
    elif test_or_train_data == test:
        return input_lang, output_lang, test_dataloader

In [139]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, encoder_scheduler, decoder_scheduler,
          criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

       

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        encoder_scheduler.step()
        decoder_scheduler.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [140]:
def test_epoch(dataloader, encoder, decoder, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [141]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [142]:
def test(test_dataloader, encoder, decoder, n_epochs, print_every=1, plot_every=1, evaluateRandomly=True):
    encoder.eval()
    decoder.eval()
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    correct_tokens = 0
    total_tokens = 0

 
    criterion = nn.NLLLoss()
    with torch.no_grad():
        
        for epoch in range(1, n_epochs + 1):
            loss = test_epoch(train_dataloader, encoder, decoder, criterion)
            print_loss_total += loss
            plot_loss_total += loss
            
            if epoch % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

            if epoch % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
            
            if evaluateRandomly==True:
                for i in range(10):
                    pair = random.choice(pairs)
                    print('>', pair[0])
                    print('=', pair[1])
                    output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
                    output_sentence = ' '.join(output_words)
                    print('<', output_sentence)
                    print('')

In [143]:
def train1(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.0001, weight_decay=1e-6,
               print_every=1, plot_every=1):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
    encoder_scheduler = optim.lr_scheduler.StepLR(encoder_optimizer, step_size=20, gamma=0.8)
    decoder_scheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size=20, gamma=0.8)
    
    
    
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, encoder_scheduler,
                           decoder_scheduler, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [144]:
def train2(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, weight_decay=1e-5,
               print_every=1, plot_every=1):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
    encoder_scheduler = optim.lr_scheduler.StepLR(encoder_optimizer, step_size=30, gamma=0.95)
    decoder_scheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size=30, gamma=0.95)
    
    
    
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, encoder_scheduler,
                           decoder_scheduler, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [145]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [146]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, sentence)

    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()

    decoded_words = []
    for idx in decoded_ids:
        if idx.item() == EOS_token:
            decoded_words.append('<EOS>')
            break
        decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [147]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [148]:
# Training Model 1

hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size, train1)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train1(train_dataloader, encoder, decoder, 20, print_every=1, plot_every=1)

Reading lines...
Read 140 sentence pairs
Trimmed to 87 sentence pairs
Counting words...
Counted words:
prior 716
last 733
0m 12s (- 4m 5s) (1 5%) 6.8065
0m 22s (- 3m 20s) (2 10%) 6.5207
0m 31s (- 2m 58s) (3 15%) 6.2379
0m 41s (- 2m 45s) (4 20%) 5.9574
0m 50s (- 2m 32s) (5 25%) 5.6690
1m 0s (- 2m 20s) (6 30%) 5.3966
1m 9s (- 2m 9s) (7 35%) 5.1022
1m 18s (- 1m 57s) (8 40%) 4.8790
1m 28s (- 1m 47s) (9 45%) 4.6404
1m 37s (- 1m 37s) (10 50%) 4.4580
1m 47s (- 1m 27s) (11 55%) 4.2396
1m 56s (- 1m 17s) (12 60%) 4.0598
2m 6s (- 1m 8s) (13 65%) 3.8386
2m 15s (- 0m 58s) (14 70%) 3.6453
2m 26s (- 0m 48s) (15 75%) 3.5304
2m 35s (- 0m 38s) (16 80%) 3.4009
2m 45s (- 0m 29s) (17 85%) 3.2684
2m 55s (- 0m 19s) (18 90%) 3.1309
3m 5s (- 0m 9s) (19 95%) 3.0585
3m 15s (- 0m 0s) (20 100%) 2.8957


In [149]:
#Testing Model 1

hidden_size = 128
batch_size = 32

input_lang, output_lang, test_dataloader = get_dataloader(batch_size, test)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

test(test_dataloader, encoder, decoder, 20, print_every=1, plot_every=1, evaluateRandomly=True)

Reading lines...
Read 140 sentence pairs
Trimmed to 87 sentence pairs
Counting words...
Counted words:
prior 716
last 733
0m 3s (- 1m 12s) (1 5%) 6.8819
> K9189 N360 Y842 Y92230 Z21 Z8546 F17210
= A047 K55029 D65 J9601 R6521 K7200 A419 N179 E872 M79A3 D62 F05 N360 Y842 Z66 Z515 Z21 I468 I10 E875 E162 E8339 E8351 F17210 Z8546 Z933
< T50995A Z923 J9690 G936 A0471 B690 J984 I9751 B690 I9751 K560 Y92000 Z992 E1122 E1122 I272 A408 A4101 Z833 L89610 J939 E1169 Z85038 F05 D709 M4712 I428 A408 Z6825 I25810 Z8701 R402322 C715 Z8572 L89610 L89153 R0489 K810 R1310 A408 I96 H5462 T8119XA Z1611 N183 N183 I2699 R1310 J441 F79 Y92019 S2243XA Y92019 H409 H409 G936 A0471 B690 J984 T508X5A I272 I2510 L98429 K651 K651 K651 R9401 B258 I69122 Z95810 Z6825 E039 D696 Z8701 K56609 R402322 B690 J984 I9751 B690 I9751 K560 Y92000 J15211 L02213 Y92234 Y92234 J90 Z923 J9690 R64 C250 M79A3 M79A3 E8809 B690 J984 I9751 B690 J984 I9751 B690 I9751 K560 Y92000 Z992 E1122 E1122 I272 A408 A4101 Z833 L89610 J939 E1169 Z850

< S12110A N5089 S0511XA S0511XA I272 A408 M8008XD I482 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 F329 M8008XD R200 M868X7 

< I9752 I77810 C781 E1169 Z8572 D8689 I214 A499 I272 A408 M8008XD I63231 A4101 R571 Z8572 D8689 R740 R34 R34 K224 K224 K224 H5462 H5462 W5501XA T68XXXA B007 B007 I700 R000 R000 R000 R000 M4640 D689 Z8501 A4101 R571 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 R4020 A4189 C781 R739 I25810 D6481 D6481 W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA W2209XA

< I9752 I77810 C781 J930 T508X5A R471 Z6825 I25810 T17810A Z87891 Z8529 Z8529 I25118 B20 K5641 G300 C7989 E8771 Z933 Y92480 F329 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I272 M8008XD A408 M8008XD I63231 G3183 M1A9XX0 Z85828 K3184 K3184 R34 C9200 G4700 I132 T508X5A F329 Z9981 I481 I

< S12110A G0490 E039 I82432 C9000 H548 H548 H548 Z6826 J910 F79 I998 E538 W06XXXA C7800 T508X5A R579 Z8701 I480 J151 J189 I82442 I951 S2243XA E222 R402212 G40901 G40901 D472 Z8701 K56609 G9341 J151 J189 H269 Z6832 A499 R1310 A408 M8008XD I63231 Y92230 T451X5A R402212 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T17210A F79 E861 R041 R041 R041 S2243XA T68XXXA G9382 Z85828 Z923 E538 W06XXXA C7800 T17210A T1721

...

< I9752 I77810 R578 R4020 R259 B007 B007 I700 R000 R000 R000 R000 R000 J151 J189 H269 Z6832 I339 S0511XA S0511XA S0511XA R9401 D8689 I272 A408 M8008XD A4101 C155 I509 I2721 J189 H269 E8771 Z85820 Z833 S12600A Z947 M19019 Y92230 R041 R041 S2243XA T68XXXA B007 I700 I481 I2721 T82868A F05 Z8701 K56609 R402322 J189 H269 Z6832 R51 S1191XA I272 M8008XD E8771 Z933 Y92480 I2699 K8010 F09 I249 S1191XA I4901 I272 M8008XD R634 Z9049 Y848 R200 E113499 R471 Z6825 I25810 R32 Z8701 K56609 R918 W19XXXA K7030 H269 K7290 I272 I2510 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 R000 R000 G0490 R000 R000 R000 

< I9752 I77810 C781 I480 J930 F1910 C155 K562 K228 R238 T8119XA D638 D638 R502 A0471 M868X7 F329 R51 K7030 E119 L89153 E222 R402212 K810 G935 Z66 I70201 S0511XA S0511XA R29703 W010XXA E1169 S22089A Z23 Z23 R0489 I6310 S12030A E1065 D696 E1065 K761 K7030 I5022 E039 G935 G935 Z66 I70201 R32 E519 K761 K761 R29711 Z6832 A499 R1310 Z954 J159 Y92019 S2243XA C259 R4789 X58XXXA X58XXXA E10649 Z8551 R4789 X58XXXA X58XXXA C9200 X58XXXA C9200 I132 Z8673 Z1624 Z1624 R339 Z931 E11319 E873 Z96643 R571 T451X5A E1151 Y92234 J90 Z923 J9690 R64 E042 I6302 Z85118 I82441 I69351 L89153 G92 S12110A E222 R402212 K810 Z8542 E113499 I712 R40244 R40244 B007 I2584 J15211 D696 Z8701 K56609 G9341 R40244 B007 I2584 J15211 D696 Z8701 K56609 G9341 R579 L98429 I509 Z8501 A4101 E780 R571 T451X5A E519 E861 R740 K7030 D539 R041 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R041 R041 R319 R

< I9752 I77810 C781 J930 T508X5A I272 I2510 B007 B007 I700 R000 R000 R000 R000 F05 B258 Z992 Z8529 Z8529 I25118 Z8529 I25118 B20 K5641 H269 Z992 T8119XA D638 D638 Z66 H3530 H3530 C50911 E8339 Z96643 M4640 E874 D649 I25118 T380X5A G893 I2510 I2129 M79A3 I252 E873 Z96643 Z21 R6521 E8770 M79A3 I871 M6518 I871 D638 R041 R041 R041 S2243XA Z86010 N170 D684 D6959 I2129 A4101 Z6831 R4020 E10649 K0889 L97429 N471 I272 A408 M8008XD I63231 A4101 I350 G9589 Z933 R34 K228 I2119 R0489 B007 B007 I63231 A4101 E780 E8352 Z950 D696 Z8701 H409 H409 Z96642 I2129 I2129 I63231 I350 R4020 A0471 B690 J984 D638 D638 Y842 Z923 E538 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 J15211 Y92234 Y92234 J90 Z923 E538 Y92234 J90 Z923 E538 J

< I9752 I77810 C781 M868X8 A4101 R571 B182 Z8701 G9341 B007 I82532 H269 Z6832 A499 R1310 Z954 J159 Z947 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H3530 Z66 H3530 H3530 C50911 Y848 B182 D6481 I871 D638 Z66 H3530 H3530 C50911 I350 Z8572 Z947 Y92230 R740 Z933 R34 K228 R34 G935 Z66 H353

< I9752 I77810 M868X8 N360 I2584 I63231 G3183 M1A9XX0 E883 H548 H548 G893 K3184 R34 K7469 L98429 B690 I132 Z8619 F329 K56609 I2720 G931 G931 R4020 I951 J9610 J984 R0489 B007 B007 I82532 I700 G8929 Z85828 M79A3 C7800 I252 K7581 C7800 M868X7 M79A3 R55 I4891 E042 K55029 Z6826 Z6826 J910 F79 I998 F1910 B9561 J150 D8689 C9000 W06XXXA C7800 C9000 I4891 D709 I4891 G0490 R892 C8308 T451X5A I081 T451X5A L89610 L89153 R0489 K810 Z87891 R34 G935 R40244 E883 I951 J9610 J984 R0489 B007 B007 I82532 I700 G8929 Z85828 M79A3 C7800 I252 K7581 C7800 M868X7 M79A3 R55 I4891 E042 K55029 Z6826 Z6826 J910 F79 I998 F1910 B9561 J150 D8689 C9000 W06XXXA C7800 C9000 I4891 D709 I4891 G0490 R892 C8308 T451X5A I081 T451X5A L89610 L89153 R0489 K810 Z87891 R34 G935 R40244 E883 I951 J9610 J984 R0489 B007 B007 I82532 I700 G8929 Z85828 M79A3 C7800 I252 K7581 C7800 M868X7 M79A3 R55 I4891 E042 K55029 Z6826 Z6826 J910 F79 I998 F1910 B9561 J150 D8689 C9000 W06XXXA C7800 C9000 I4891 D709 I4891 G0490 R892 C8308 T451X5A I081 T4

In [150]:
#Training Model 2

hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size, train2)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train2(train_dataloader, encoder, decoder, 20, print_every=1, plot_every=1)

Reading lines...
Read 140 sentence pairs
Trimmed to 87 sentence pairs
Counting words...
Counted words:
prior 716
last 733
0m 10s (- 3m 26s) (1 5%) 5.4252
0m 21s (- 3m 15s) (2 10%) 2.9905
0m 32s (- 3m 4s) (3 15%) 1.7612
0m 43s (- 2m 54s) (4 20%) 1.1825
0m 55s (- 2m 45s) (5 25%) 0.9675
1m 6s (- 2m 35s) (6 30%) 0.9288
1m 17s (- 2m 24s) (7 35%) 0.8670
1m 29s (- 2m 14s) (8 40%) 0.7947
1m 41s (- 2m 3s) (9 45%) 0.7537
1m 53s (- 1m 53s) (10 50%) 0.6929
2m 5s (- 1m 42s) (11 55%) 0.6525
2m 17s (- 1m 31s) (12 60%) 0.6896
2m 29s (- 1m 20s) (13 65%) 0.6504
2m 41s (- 1m 9s) (14 70%) 0.6134
2m 53s (- 0m 57s) (15 75%) 0.6169
3m 5s (- 0m 46s) (16 80%) 0.6493
3m 17s (- 0m 34s) (17 85%) 0.6643
3m 29s (- 0m 23s) (18 90%) 0.6222
3m 42s (- 0m 11s) (19 95%) 0.6261
3m 55s (- 0m 0s) (20 100%) 0.6385


In [151]:
#Testing Model 2

hidden_size = 128
batch_size = 32

input_lang, output_lang, test_dataloader = get_dataloader(batch_size, test)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

test(test_dataloader, encoder, decoder, 20, print_every=1, plot_every=1, evaluateRandomly=True)

Reading lines...
Read 140 sentence pairs
Trimmed to 87 sentence pairs
Counting words...
Counted words:
prior 716
last 733
0m 4s (- 1m 31s) (1 5%) 6.8143
> G935 G950 I252 N400 V499XXS Z9181
= G935 J9600 G9589 D62 Z66 R1310 Q758 I469 D72829 N401 R338 Z87891
< I5023 Y838 Z923 I4581 R34 R471 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J704 J

< I5023 Y838 Z923 I4581 R34 R471 K311 R402112 D539 I82412 I130 I82412 I130 I82412 J80 R197 J80 R197 H409 H409 I63512 I63512 I63512 Z833 K6289 K6289 J159 K9189 K5720 F05 R579 I2729 I2729 I452 I2729 I452 I2729 I2729 I452 D8689 D8689 D8689 F419 M4640 F840 J918 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 K6289 J159 K9189 K5720 F05 R579 I2729 I2729 I452 I2729 I452 I2729 I2729 I452 D8689 D8689 D8689 F419 M4640 F840 J918 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 K6289 J159 K9189 K5720 F05 R579 I2729 I2729 I452 I2729 I452 I2729 I2729 I452 D8689 D8689 D8689 F419 M4640 F840 J918 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 K6289 J159 K9189 K5720 F05 R579 I2729 I2729 I452 I2729 I452 I2729 I2729 I452 D8689 D8689 D8689 F419 M4640 F840 J918 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 K6289 J159 K9189 K5720 F05 R579 I2729 I2729 I452 I2729

< W2209XA G902 G902 I728 Z45018 K311 R29711 J988 J988 E10649 Z833 R29711 J988 E10649 Z833 R29711 B440 M797 I63231 K5790 G8929 G8929 S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S2241XA S224

< C786 Z98890 Z98890 Z8673 H5461 B952 F17290 F17290 I69320 N179 I5030 S66822A K7290 E039 Z7901 Z6832 E46 K763 F1020 Z96653 D500 M4640 F840 <EOS>

0m 18s (- 1m 43s) (3 15%) 6.8129
> D481 K921 A048 D62 G4733 N400 R911 E785 K2950 Z87891 Z800
= D481 T8119XA J9601 K91840 D62 I469 Y838 Y92230 E785 E669 Z6829 M6530 G4733 Z87891 D649
< L89610 J9600 D539 I82412 I130 I82412 I130 I82412 Z7901 Z45018 S72001A Q758 D8689 D8689 J440 I5030 I358 B182 I358 B182 I358 B182 H5461 I214 I63512 I63512 I63512 I63512 Z833 K6289 I5030 Z7901 G8929 G8929 K5720 Z96653 Z6832 D500 Z1611 F840 Z930 B182 Z006 C781 K5900 Z1611 F840 J918 I69391 F0390 D709 I5023 Y838 Z923 Z923 I4581 R34 R471 K311 W2209XA G902 K2270 K311 Z9221 I69391 I69391 E561 S2243XA I272 G893 H3530 E876 R402112 M797 I63231 I63512 I63512 Z833 K6289 I5030 E2740 R197 Z8546 E11319 I70212 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 Z1611 F840 Z930 B182 Z006 C781 K5900 Z1611 F840 J918 I69391 F0390 D709 I5023 Y838 Z923 Z923 I4581 R34 R471 K311 W2209XA G902

< W2209XA G902 G902 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I70212 D638 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 R791 K7200 Y834 Z6827 J159 I7

...

< I639 I5023 Y838 Z923 I4581 Z950 Y836 I871 J09X2 J441 R29703 K7290 E222 E039 N170 Z96643 I2729 I2729 I452 I2729 I452 D8689 D8689 F419 M4640 F840 J918 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 I5030 E2740 R197 Z8546 E11319 I70212 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 E46 R402112 I339 I339 I638 K2270 W19XXXA R197 D539 I82412 I130 I82412 I130 I82412 Z7901 Z45018 S72001A Q758 D8689 D8689 J440 I5030 I358 S069X0S I358 B182 H5461 I214 I82441 F39 T380X5A Z833 K7290 T68XXXA D6832 S0101XA J440 I70201 J440 S36029A I132 J860 G902 K2270 W19XXXA I452 M4802 I69320 N179 Z833 S1191XA I5030 I824Z1 Y832 N400 Z96643 I2729 I2729 I452 D8689 D8689 D8689 F419 M4640 Z85028 Z833 S1191XA I70201 I69391 I69391 E561 S2243XA C784 I10 I071 J159 J159 I69351 I69391 I69391 G20 I63512 I63512 I63512 I63512 I63512 I63512 I63512 I63512 Z833 K6289 I5030 E2740 R197 Z8546 E11319 I70212 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 D500 H5461 B952 E46 R402112 I339 I3

< E7800 E7800 E7800 M6530 G936 Z66 Z66 S12030A Z66 E222 S36112A Z96643 R1011 J9600 Y836 H269 F17290 F17290 I69320 N179 B007 J449 D500 Z21 F840 A408 Z96653 Y92238 T380X5A S1191XA M5489 K311 M4802 N401 Z87440 K7200 K7200 M797 I63231 M86172 J982 M86172 W06XXXA Z96643 Y92019 Y832 T17890A B9689 E113499 I82441 F88 R188 W06XXXA Z833 W06XXXA Z833 W06XXXA Z833 I50811 R112 D539 I82412 I130 I82412 I130 I82412 I130 Z4502 Z794 R402432 I69351 R4789 R4789 Z7901 Z6832 I70212 B9561 I358 I358 B182 I358 B182 I358 B182 H5461 I214 I82441 F39 T380X5A Z833 I50811 R112 L089 I2129 Z993 R001 K761 Z96643 Y92019 Z006 K652 K5720 I130 N400 S2241XA S2241XA I82411 Z8701 Z9181 F1020 G620 T451X1A Z96643 Z8542 K7010 I63512 I63512 Z833 K6289 I5030 W06XXXA F1123 G902 K2270 W19XXXA R197 D539 I82412 I130 I82412 I130 I82412 J80 Z9049 E561 K208 W109XXA R197 Z45018 M4802 I340 E6601 R0902 I82411 K6289 K6289 I5030 I638 I319 R531 G931 F0390 K652 M61451 R64 C782 Z85048 R471 R791 R0902 G902 K2270 F419 C784 R319 T380X5A A419 Z95810 

< I5023 Y838 Z923 I4581 R34 R471 K311 W2209XA G902 K2270 F419 C784 R319 T380X5A A419 A419 W010XXA Z66 Z66 I2720 Z96643 I63512 I63512 I63512 I63512 Z833 K6289 I5030 E2740 R197 Z8546 E11319 I70212 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E46 F1910 G902 G8929 G8929 G8929 G8929 K5720 Z96653 Z6832 E4

< C786 Z98890 Z98890 N186 Z902 Z902 I824Z1 N186 Z902 I824Z1 I824Z1 J156 S72001A Q758 S0511XA S0511XA T17810A R4789 R4789 R4789 Z7901 Z96653 Z6832 I70212 I358 I358 I358 W2209XA J156 D61810 F17210 E561 E10649 M61451 I871 S0101XA J440 Z96653 F1910 C7902 R1011 J449 F39 T380X5A G8191 A419 A419 R410 K311 G4089 S22089A J860 E039 B952 N170 R112 Z96643 K7469 A4181 J151 Z98890 A4181 B952 E46 F1910 G902 K311 W2209XA G902 G8929 G8929 G8929 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G8929 G8929 G8929 G8929 K5720 Z96653 G8929 G89

< I5023 Y838 Z923 I4581 Z950 Y836 H269 F17290 F17290 I69320 N179 B007 J449 D7589 K7200 I491 K6289 I5030 G3183 D638 S0511XA S0511XA J942 W5501XA Z1621 Z006 W010XXA E561 K208 M21371 Z902 Z902 I638 Z8673 Z6834 F840 <EOS>

